# DS PROJECT - Predicting the outcome of driving exams in Estonia

## Reading and examining the data

### Reading and formatting

In [31]:
import pandas as pd

# Read the three CSV files into Pandas DataFrames

data2021 = pd.read_csv("se_2021.csv")
data2022 = pd.read_csv("se_2022.csv")
data2023 = pd.read_csv("se_2023.csv")

# Combine the three DataFrames
combined_data = pd.concat([data2021, data2022, data2023], axis=0, ignore_index=True)

#split KUUPAEV into AASTA and KUU
combined_data[['AASTA', 'KUU']] = combined_data['KUUPAEV'].str.split('-', expand=True)
combined_data['AASTA'] = pd.to_numeric(combined_data['AASTA'])
combined_data['KUU'] = pd.to_numeric(combined_data['KUU'])

#drop KUUPAEV
combined_data = combined_data.drop(columns=['KUUPAEV'])
combined_copy = combined_data.copy()

combined_data.head()

,EKSAMI_SOORITAJA,BYROO,KATEGOORIA,ERITINGIMUSED,VIIMANE_AUTOKOOL,SOIDUOPETAJA_KAASAS,EKSAMINEERIJA,SEISUND,KESTUS,KATK_POHJUS,MITTEARVESTATUD,VEAD,AASTA,KUU
0,fd4a476c2fff5845,Haapsalu,B,101,osaühing KLIMBERG,E,31451a368e962058,SOORITATUD,50.0,NaN,NaN,NaN,2021,1
1,8fe3816987e62971,Jõgeva,B,101,OÜ AUTOSÕIT,E,ae264be409f2ca9f,MITTE_SOORITATUD,24.0,NaN,EESOIGUS,NaN,2021,1
2,1131ad82d5710721,Põlva,B,101,H&M MERTENS OÜ,E,40bcf2cea820fca6,MITTE_SOORITATUD,24.0,NaN,EESOIGUS,NaN,2021,1
3,819bfde54073d8e7,Võru,B,101,Tugev Partner OÜ,E,bf2b7218d36e2a3d,MITTE_SOORITATUD,48.0,NaN,JALGIMINE|LIIKLUSOHUTUS|TEELPUSIMINE,NaN,2021,1
4,06493201c082d7f2,Haapsalu,B,101,OÜ Raak Company,E,31451a368e962058,SOORITATUD,51.0,NaN,NaN,NaN,2021,1


### Looking at values in the columns and encoding categorical variables into numerical representations

In [21]:
combined_data['BYROO'].value_counts()

Tallinn       49056
Tartu         15323
Rakvere       10772
Pärnu          9759
Jõhvi          8623
Narva          6262
Paide          6041
Viljandi       5306
Rapla          4097
Haapsalu       3709
Võru           3446
Kuressaare     3232
Valga          1052
Jõgeva          912
Põlva           907
Kärdla          718
Saue              2
Name: BYROO, dtype: int64

In [ ]:
combined_copy #one-hot

In [23]:
combined_data['KATEGOORIA'].value_counts()

B      91552
C       9684
A       8310
CE      7225
BE      3834
T       2599
D       1920
AM      1678
A2      1377
A1       836
C1       175
Z         22
C1E        5
Name: KATEGOORIA, dtype: int64

In [ ]:
combined_copy #one-hot

In [24]:
combined_data['ERITINGIMUSED'].value_counts() #uuri 101|78|78, kui ei saa selgust drop, neid ainult 3

101          69243
101|78       19195
79.01         1331
96            1091
78             982
79.02          347
106             13
101|78|78        3
Name: ERITINGIMUSED, dtype: int64

In [ ]:
combined_copy #one-hot

In [28]:
value_counts_all = combined_data['VIIMANE_AUTOKOOL'].value_counts().sort_index()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(value_counts_all)

"Autom" Põltsamaa Liiklusklubi               225
A&J AUS OÜ                                     9
A+ AUTOKOOL OÜ                               256
A-AUTOM, OÜ                                   23
A2 autokool OÜ                                27
AB Testudo OÜ                                 61
AKTSIASELTS PÄRNU ATP                          3
ALEKSANDRI AUTOKOOL                            3
ALFA PRO OÜ                                   75
ALFAWING OÜ                                   78
ANDREI PERVORODNÕI - Stendall                  9
ARION OÜ                                     539
AUTOKOOLITUS OÜ                              475
AUTOREM, OÜ                                   20
AV Rae Liikluskoolituse OÜ                   581
Aabe Cool UÜ                                   5
Abja Gümnaasium                               75
Aire Tammik Autokool OÜ                      628
Aivar Tärk Autokool                           71
Aktsiaselts Odium                           5133
Aktsiaselts Tallinna

In [ ]:
combined_copy #vaata sarnased ja lühendid läbi, kõik variandid osaühingutest jms lühenditeks nagu OÜ, one-hot

In [32]:
combined_data['SOIDUOPETAJA_KAASAS'].value_counts()

E    119935
J      5758
Name: SOIDUOPETAJA_KAASAS, dtype: int64

In [ ]:
combined_copy #binaarne

In [33]:
combined_data['EKSAMI_SOORITAJA'].value_counts()

7da809d9d7b050c3    18
293e4a0f5ded2650    17
8f5fc6fb5d3f10a2    16
9389bc6e7479bc10    16
2e5a49e327ad8546    16
                    ..
ee1f85ae4d8e7d75     1
9727e3ffd56f6da2     1
b721a3b4b7e5f826     1
836ec1b5c8bb5727     1
32426dfae15e014f     1
Name: EKSAMI_SOORITAJA, Length: 68303, dtype: int64

In [ ]:
combined_copy #new column for how many times has taken exam (think through how to implement), after drop eksami_sooritaja

In [40]:
combined_data[['EKSAMINEERIJA','AASTA']].value_counts().sort_index()

EKSAMINEERIJA     AASTA
00fe4a180f5c474e  2021       40
                  2023        2
075e15cf091d347b  2021     1143
                  2022     1225
                  2023      897
                           ... 
faeaf99580d6be30  2022     1177
                  2023      448
ff4d9b9b519322e3  2021     1277
                  2022     1261
                  2023      979
Length: 133, dtype: int64

In [ ]:
combined_copy #one-hot

In [41]:
combined_data['SEISUND'].value_counts()

SOORITATUD           70087
MITTE_SOORITATUD     55298
EI_ILMUNUD_KOHALE     2604
KATKESTATUD           1228
Name: SEISUND, dtype: int64

In [ ]:
combined_copy #muuda numbriteks

In [43]:
combined_data['KESTUS'].describe()

count    125414.000000
mean         39.512088
std          18.024153
min           0.000000
25%          27.000000
50%          43.000000
75%          51.000000
max        1024.000000
Name: KESTUS, dtype: float64